# Load Two Experiment Results For Comparison

This notebook takes two results folders generateed from `partition_image_bm.ipynb` and compares them against eachother. The purpose is to compare the performance differences between that of an experiment executed in a AI Kit Conda environment and the other in a stock (default) Conda channel environment (where all the libraries are from the Conda/pip channels and not Intel Conda channels). There are three plots comparing the four different runs executed from the two results folders. Each results folder contains an Intel-optimized (oneDNN flags are set to on) and a non-optimized (oneDNN flags are set to off) experiment.

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# color to use for AI Kit
intel_blue = '#0071c5'

# Set the path to your results folders
stock_env_results_dir = 'your/stock/results/folder'
aikit_env_results_dir = 'your/aikit/results/folder'

### Load results
A total of 4 DataFrames are created from the two experiment results folders where each folder contains oneDNN on (oneDNN_off_times_aggregated.csv) and off (oneDNN_on_times_aggregated.csv) results.

In [ ]:
stock_off_df = pd.read_csv(os.path.join(stock_env_results_dir,'oneDNN_off_times_aggregated.csv'), index_col=0)
stock_on_df = pd.read_csv(os.path.join(stock_env_results_dir, 'oneDNN_on_times_aggregated.csv'), index_col=0)
intel_off_df = pd.read_csv(os.path.join(aikit_env_results_dir, 'oneDNN_off_times_aggregated.csv'), index_col=0)
intel_on_df = pd.read_csv(os.path.join(aikit_env_results_dir, 'oneDNN_on_times_aggregated.csv'), index_col=0)

### Plot the experiment averages against each other
Line plot all 4 experiment averages along with their confidence intervals with respect to max_intervals.

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
x = stock_on_df.index
ax.plot(x, stock_on_df['mean'],  marker='.', color='r', label='oneDNN Stock')
ax.fill_between(
    x, stock_on_df['ci_lower'], stock_on_df['ci_upper'], color='r', alpha=.1)

ax.plot(x, stock_off_df['mean'], color='r', marker='.', linestyle='--', label='Stock')
ax.fill_between(
    x, stock_off_df['ci_lower'], stock_off_df['ci_upper'], color='r', alpha=.1)



ax.plot(x, intel_on_df['mean'], color=intel_blue, marker='.', label='oneDNN AI Kit')
ax.fill_between(
    x, intel_on_df['ci_lower'], intel_on_df['ci_upper'], color=intel_blue, alpha=.1)

ax.plot(x, intel_off_df['mean'], color=intel_blue, marker='.',  linestyle='--', label='AI Kit')
ax.fill_between(
    x, intel_off_df['ci_lower'], intel_off_df['ci_upper'], color=intel_blue, alpha=.1)



ax.set_ylim(ymin=0)
ax.set_xlim(xmin=64, xmax=2048)
ax.set_xticks(x)
ax.set_ylabel('Time (s)')
ax.set_xlabel('Max Evaluations')
ax.set_title('Avg Compute Time by Max Evaluations (n=5)')
ax.grid(axis='y')
ax.legend()

fig.autofmt_xdate(rotation=45)

Bar plot all 4 experiment averages with respect max_evals.

In [ ]:
# set width of bar
barWidth = 0.22
fig = plt.subplots(figsize =(14, 7))

# Set position of bar on X axis
br1 = np.arange(6)
br2 = [x + barWidth for x in br1]
br3 = [x + barWidth for x in br2]
br4 = [x + barWidth for x in br3]
 
# Make the plot
plt.bar(br1, stock_off_df['mean'], color='r', hatch='x', width = barWidth,
        edgecolor='white', label='Stock')
plt.bar(br2, stock_on_df['mean'], color='r', width = barWidth,
        edgecolor='white', label='OneDNN Stock')
plt.bar(br3, intel_off_df['mean'], color=intel_blue, hatch='x', width = barWidth,
        edgecolor='white', label='AI Kit')
plt.bar(br4, intel_on_df['mean'], color=intel_blue, width = barWidth,
        edgecolor='white', label='OneDNN AI Kit')
 
# Adding Xticks
plt.xlabel('Max Evals')
plt.ylabel('Average Compute Time (s)')
plt.xticks([r + barWidth for r in range(6)],
        ['64', '128', '256', '512', '1024', '2048'])
plt.title('Average Compute Times accross All Experiments', fontsize=16)

# Adding value labels to top of bars
for bar, means in zip([br1, br2, br3, br4], [stock_off_df['mean'], stock_on_df['mean'], intel_off_df['mean'], intel_on_df['mean']]):
    for col, mean in zip(bar, means.tolist()):
        plt.text(col, mean, str(round(mean, 1)), ha='center')

plt.legend()
plt.show()

Bar plot the percent decrease between the stock environment with oneDNN off and AI Kit environment with oneDNN set to on. These two experiments should show the greatest difference in computation time due to the Intel optimizations.

In [ ]:
# Get percent decrease
diffs = []

for on, off in zip(intel_on_df['mean'], stock_off_df['mean']):
    diffs.append(((on - off)/off)*100)

# compare reduction in time between stock and Intel optimizations
diffs_series = pd.Series(np.array(diffs)*-1)
plt.figure(figsize=(10,6))
fig = diffs_series.plot(kind='bar')
fig.set_xticklabels(['64', '128', '256', '512', '1028', '2048'])
fig.bar_label(fig.containers[0], label_type='edge')
fig.set_title('Pure Stock VS OneDNN AI Kit Percent Decrease in Computation Time')
fig.set_xlabel('Max Evaluations')
fig.set_ylabel('% Decrease')